# Task space inverse dynamics control
## Point-to-point

In [ ]:
import numpy as np
from simulator import Simulator
from pathlib import Path
from typing import Dict
import os 
import pinocchio as pin
import matplotlib.pyplot as plt
from IPython.display import Video

In [ ]:
current_dir = os.getcwd()
xml_path = os.path.join(current_dir, "robots/universal_robots_ur5e/ur5e.xml")
model = pin.buildModelFromMJCF(xml_path)
data = model.createData()

Для того, чтобы сделать TSID контроллер, неохбодимо сделать 2 контура обратных связей. Первый переводит $\mathbf{X^d} \rightarrow \mathbf{a_q}$ (outer loop), а второй контур - $\mathbf{a_q} \rightarrow \mathbf{u}$ (inner loop)

Первый контур:

\begin{equation}
a_X=\ddot{X}^d - k_p(X - X^d) - k_d(\dot{X}-\dot{X}^d)
\end{equation}

\begin{equation}
a_q = J^{-1}(a_X - \dot{J}\dot{q})
\end{equation}

Второй контур:

\begin{equation}
u = \mathbf{M(q)}a_q + \mathbf{C(q,\dot{q})}\dot{q} + \mathbf{g(q)}
\end{equation}


При этом $\mathbf{X} \in \mathbb{R}^6$ 3 линейные координаты и 3 угловые, для составления ошибки по вращению (3-х мерный вектор) использована следующая формула:

\begin{equation}
\mathbf{\tilde{S}} = -log(\mathbf{R^d R^T})
\end{equation}

Для ошибки по положению:

\begin{equation}
\mathbf{\tilde{p}} = \mathbf{p} - \mathbf{p^d}
\end{equation}

In [ ]:
def task_space_p2p_controller(q: np.ndarray, dq: np.ndarray, t: float, desired: Dict) -> np.ndarray:
    
    #PD coeff
    kp = np.array([200, 200, 200, 200, 200, 200])*np.eye(6) 
    kd = np.array([100, 100, 100, 100, 100, 100])*np.eye(6)

    # Convert desired pose to SE3
    desired_position = desired['pos']
    desired_quaternion = desired['quat'] # [w, x, y, z] in MuJoCo format
    desired_quaternion_pin = np.array([*desired_quaternion[1:], desired_quaternion[0]]) # Convert to [x,y,z,w] for Pinocchio
    # Convert to pose and SE3
    desired_pose = np.concatenate([desired_position, desired_quaternion_pin])
    desired_se3 = pin.XYZQUATToSE3(desired_pose)

    #desired parameters
    R_des = desired_se3.rotation
    p_des = desired_se3.translation
    ddX_des = np.zeros(6)
    dX_des = np.zeros(6)

    # Computations
    pin.computeAllTerms(model, data, q, dq)
    pin.forwardKinematics(model, data, q, dq)

    # Get the frame pose
    ee_frame_id = model.getFrameId("end_effector")
    ee_pose = data.oMf[ee_frame_id]
    ee_position = ee_pose.translation
    ee_rotation = ee_pose.rotation

    # Get velocities, accelerations and Jacobians
    frame = pin.WORLD
    twist = pin.getFrameVelocity(model, data, ee_frame_id, frame).vector
    dtwist = pin.getFrameAcceleration(model, data, ee_frame_id, frame)
    J = pin.getFrameJacobian(model, data, ee_frame_id, frame)
    dJ = pin.computeJointJacobiansTimeVariation(model, data, q, dq)

    # State error
    error_rot = -pin.log3(R_des @ ee_rotation.T)
    X_error = np.hstack([ee_position - p_des, error_rot])

    # Outer loop and inner loop
    a_x = ddX_des - kp @ X_error - kd @ (twist - dX_des)
    a_q = np.linalg.inv(J)@(a_x - dJ@dq)
    tau = data.M @ a_q + data.nle

    # logs
    print(f'p_desired : {p_des}')
    print(f'R_desired : {R_des}')
    print(f'p_end_effecor : {ee_position}')
    print(f'R_end_effecor : {ee_rotation}')
    print(f'X_error : {X_error}')

    return tau

Запуск симуляции

In [ ]:
# Create logging directories
Path("logs/videos").mkdir(parents=True, exist_ok=True)

print("\nRunning task space controller...")
sim = Simulator(
    xml_path="robots/universal_robots_ur5e/scene.xml",
    enable_task_space=True,
    show_viewer=True,
    record_video=True,
    video_path="logs/videos/05_hw_task_space.mp4",
    fps=30,
    width=1920,
    height=1080
)

sim.set_controller(task_space_p2p_controller)
sim.run(time_limit=10.0)

times = np.array(sim.times)
positions = np.array(sim.positions)
velocities = np.array(sim.velocities)

Графики положения и скоростей сочленений при движении от точки до точки

In [ ]:
# Process and save results
plt.figure(figsize=(20, 6))

# Joint positions plot
plt.subplot(121)
for i in range(positions.shape[1]):
    plt.plot(times, positions[:, i], label=f'Joint {i+1}')
plt.xlabel('Time [s]')
plt.ylabel('Joint Positions [rad]')
plt.title('Joint Positions over Time')
plt.legend()
plt.grid(True)

# Joint velocities plot
plt.subplot(122)
for i in range(velocities.shape[1]):
    plt.plot(times, velocities[:, i], label=f'Joint {i+1}')
plt.xlabel('Time [s]')
plt.ylabel('Joint Velocities [rad/s]')
plt.title('Joint Velocities over Time')
plt.legend()
plt.grid(True)
plt.savefig('logs/plots/05_hw_task_space_joint.png')

plt.show()

Видео симуляции при движении от точки до точки

In [ ]:
Video(os.path.join(current_dir, "logs/videos/05_hw_task_space.mp4"))

## Trajectory

В качестве траектории выберем окружность, зададим ее параметрически:

\begin{equation}
\begin{cases}
x = x_{center} + rad \cdot sin(\omega t)\\
y = y_{center} + rad \cdot cos(\omega t)\\
z = constant
\end{cases}
\end{equation}

При этом пусть центром круга куб, а его ориентация определит ориентацию рабочего органа

In [ ]:
def get_trajectory(p_des, t):
    """Calculating trajectory points"""
    
    X_des = np.zeros(6)
    dX_des = np.zeros(6)
    ddX_des = np.zeros(6)

    x = p_des[0] + 0.2*np.sin(1*t)
    y = p_des[1] + 0.2*np.cos(1*t)
    z = p_des[2]

    dx = 0.2*np.cos(1*t)
    dy = -0.2*np.sin(1*t)
    dz = 0

    ddx = -0.2*np.sin(1*t)
    ddy = -0.2*np.cos(1*t)
    ddz = 0
    
    X_des[:3] = np.hstack([x,y,z])
    dX_des[:3] = np.hstack([dx,dy,dz])
    ddX_des[:3] = np.hstack([ddx,ddy,ddz])
    
    return X_des, dX_des, ddX_des

In [ ]:
def task_space_traj_controller(q: np.ndarray, dq: np.ndarray, t: float, desired: Dict) -> np.ndarray:
    
    #PD coeff
    kp = np.array([200, 200, 200, 200, 200, 200])*np.eye(6)
    kd = np.array([100, 100, 100, 100, 100, 100])*np.eye(6)

    # Convert desired pose to SE3
    desired_position = desired['pos']
    desired_quaternion = desired['quat'] # [w, x, y, z] in MuJoCo format
    desired_quaternion_pin = np.array([*desired_quaternion[1:], desired_quaternion[0]]) # Convert to [x,y,z,w] for Pinocchio
    # Convert to pose and SE3
    desired_pose = np.concatenate([desired_position, desired_quaternion_pin])
    desired_se3 = pin.XYZQUATToSE3(desired_pose)
    R_des = desired_se3.rotation
    p_des = desired_se3.translation
    
    #Getting next point
    X_des, dX_des, ddX_des = get_trajectory(p_des, t)

    # Computations
    pin.computeAllTerms(model, data, q, dq)
    pin.forwardKinematics(model, data, q, dq)

    # Get the frame pose
    ee_frame_id = model.getFrameId("end_effector")
    ee_pose = data.oMf[ee_frame_id]
    ee_position = ee_pose.translation
    ee_rotation = ee_pose.rotation

    # Get velocities, accelerations and Jacobians
    frame = pin.WORLD
    twist = pin.getFrameVelocity(model, data, ee_frame_id, frame).vector
    dtwist = pin.getFrameAcceleration(model, data, ee_frame_id, frame)
    J = pin.getFrameJacobian(model, data, ee_frame_id, frame)
    dJ = pin.computeJointJacobiansTimeVariation(model, data, q, dq)

    # State error
    error_rot = -pin.log3(R_des @ ee_rotation.T)
    X_error = np.zeros(6)
    X_error[:3] = ee_position - X_des[:3]
    X_error[3:] = error_rot

    # Outer loop and inner loop
    a_x = ddX_des - kp @ X_error - kd @ (twist - dX_des)
    a_q = np.linalg.inv(J)@(a_x - dJ@dq)
    tau = data.M @ a_q + data.nle

    # logs
    print(f'X_error : {X_error}')

    return tau

Запуск симуляции

In [ ]:
print("\nRunning task space controller...")
sim = Simulator(
    xml_path="robots/universal_robots_ur5e/scene.xml",
    enable_task_space=True,
    show_viewer=True,
    record_video=True,
    video_path="logs/videos/05_hw_task_space_traj.mp4",
    fps=30,
    width=1920,
    height=1080
)

sim.set_controller(task_space_traj_controller)
sim.run(time_limit=20.0)

times = np.array(sim.times)
positions = np.array(sim.positions)
velocities = np.array(sim.velocities)

Графики положения и скоростей сочленений при движении по траектории

In [ ]:
# Process and save results
plt.figure(figsize=(20, 6))

# Joint positions plot
plt.subplot(121)
for i in range(positions.shape[1]):
    plt.plot(times, positions[:, i], label=f'Joint {i+1}')
plt.xlabel('Time [s]')
plt.ylabel('Joint Positions [rad]')
plt.title('Joint Positions over Time')
plt.legend()
plt.grid(True)

# Joint velocities plot
plt.subplot(122)
for i in range(velocities.shape[1]):
    plt.plot(times, velocities[:, i], label=f'Joint {i+1}')
plt.xlabel('Time [s]')
plt.ylabel('Joint Velocities [rad/s]')
plt.title('Joint Velocities over Time')
plt.legend()
plt.grid(True)
plt.savefig('logs/plots/05_hw_task_space_joint_traj.png')

plt.show()

Видео симуляции движения по траектории

In [ ]:
Video(os.path.join(current_dir, "logs/videos/05_hw_task_space_traj.mp4"))

Сравним траектории в проекции на плоскомть $\mathbf{XoY}$

In [ ]:
#trajectory on XY plane

x = []
y = []
z = []
x_pl = []
y_pl = []
ee_frame_id = model.getFrameId("end_effector")
for i in range(len(times)-1):
    pin.forwardKinematics(model, data, positions[i], velocities[i])
    pin.updateFramePlacement(model, data, ee_frame_id)
    ee_pose = data.oMf[ee_frame_id]
    ee_position = ee_pose.translation
    x.append(ee_position[0])
    y.append(ee_position[1])
    z.append(ee_position[2])

    x_pl.append(0.5 + 0.2*np.sin(0.0002*np.pi*i))
    y_pl.append(0 + 0.2*np.cos(0.0002*np.pi*i))


plt.figure(figsize=(10, 10))
plt.plot(x, y, 'r', label = 'real')
plt.plot(x_pl, y_pl,'--b', label = 'planned', )
plt.xlabel('X [m]')
plt.ylabel('Y [m]')
plt.title('XY trajectory')
plt.legend()
plt.grid(True)
plt.savefig('logs/plots/05_hw_task_space_xy_traj.png')
plt.show()